# 사람들 얼굴 인식해서 감정 알아내기

In [20]:
def detect_faces(path):
    """Detects faces in an image."""
    from google.cloud import vision
    import io
    client = vision.ImageAnnotatorClient() # 비전 api 객체 생성

    with io.open(path, 'rb') as image_file: # 이미지 불러오기
        content = image_file.read()

    image = vision.Image(content=content) # 이미지를 vison api에 사용 가능한 객체로 전환

    response = client.face_detection(image=image) # 생성한 객체에서 얼굴 인식 수행
    faces = response.face_annotations # 얼굴 감지의 결과를 가지고 있는 객체

    # Names of likelihood from google.cloud.vision.enums
    likelihood_name = ('UNKNOWN', 'VERY_UNLIKELY', 'UNLIKELY', 'POSSIBLE',
                       'LIKELY', 'VERY_LIKELY')
    print('Faces:')

    for face in faces:
        
        print('anger: {}'.format(likelihood_name[face.anger_likelihood])) 
        print('joy: {}'.format(likelihood_name[face.joy_likelihood]))
        print('surprise: {}'.format(likelihood_name[face.surprise_likelihood]))

        vertices = (['({},{})'.format(vertex.x, vertex.y)
                    for vertex in face.bounding_poly.vertices]) # 객체를 둘러싸는 x와 y 값 반환

        print('face bounds: {}'.format(','.join(vertices)))

    if response.error.message:
        raise Exception(
            '{}\nFor more info on error messages, check: '
            'https://cloud.google.com/apis/design/errors'.format(
                response.error.message))

In [21]:
detect_faces("C:/Users/qud97/Desktop/crawl/AutoCrawler/download/ITRI/google_0012.jpg")

Faces:
Likelihood.VERY_UNLIKELY
anger: VERY_UNLIKELY
joy: VERY_UNLIKELY
surprise: VERY_UNLIKELY
face bounds: (438,101),(485,101),(485,155),(438,155)
Likelihood.VERY_UNLIKELY
anger: VERY_UNLIKELY
joy: VERY_UNLIKELY
surprise: VERY_UNLIKELY
face bounds: (373,144),(424,144),(424,204),(373,204)
Likelihood.VERY_UNLIKELY
anger: VERY_UNLIKELY
joy: VERY_UNLIKELY
surprise: VERY_UNLIKELY
face bounds: (139,90),(182,90),(182,140),(139,140)
Likelihood.VERY_UNLIKELY
anger: VERY_UNLIKELY
joy: LIKELY
surprise: VERY_UNLIKELY
face bounds: (217,47),(255,47),(255,91),(217,91)
Likelihood.VERY_UNLIKELY
anger: VERY_UNLIKELY
joy: VERY_UNLIKELY
surprise: VERY_UNLIKELY
face bounds: (446,193),(501,193),(501,258),(446,258)
Likelihood.VERY_UNLIKELY
anger: VERY_UNLIKELY
joy: VERY_UNLIKELY
surprise: VERY_UNLIKELY
face bounds: (245,170),(287,170),(287,218),(245,218)
Likelihood.VERY_UNLIKELY
anger: VERY_UNLIKELY
joy: VERY_LIKELY
surprise: VERY_UNLIKELY
face bounds: (43,125),(84,125),(84,171),(43,171)
Likelihood.VERY_UN

# 사진에 있는 물체들 인식하기

In [9]:
def localize_objects(path):
    """Localize objects in the local image.

    Args:
    path: The path to the local file.
    """
    from google.cloud import vision
    import cv2
    client = vision.ImageAnnotatorClient()

    with open(path, 'rb') as image_file:
        content = image_file.read()
    image = vision.Image(content=content)

    objects = client.object_localization(
        image=image).localized_object_annotations # 객체, 객체의 위치, 객체가 포함된 이미지 영역의 사각형 경계에 대한 정보를 식별
    
    print('Number of objects found: {}'.format(len(objects)))
    for object_ in objects:
        print('\n{} (confidence: {})'.format(object_.name, object_.score))
        print('Normalized bounding polygon vertices: ')
        for vertex in object_.bounding_poly.normalized_vertices: # bounding_poly한 값을 원래 이미지에 대한 비율로 나타냄
            print(' - ({}, {})'.format(vertex.x, vertex.y))

# 사진을 분석해서 얼굴을 찾아내고, 얼굴에 사각형을 그려 반환하기

In [11]:
def detect_face(face_file, max_results=4):
    """Uses the Vision API to detect faces in the given file.

    Args:
        face_file: A file-like object containing an image with faces.

    Returns:
        An array of Face objects with information about the picture.
    """
    from google.cloud import vision
    client = vision.ImageAnnotatorClient()

    content = face_file.read()
    image = vision.Image(content=content)

    return client.face_detection(
        image=image, max_results=max_results).face_annotations

In [14]:
def highlight_faces(image, faces, output_filename):
    """Draws a polygon around the faces, then saves to output_filename.

    Args:
      image: a file containing the image with the faces.
      faces: a list of faces found in the file. This should be in the format
          returned by the Vision API.
      output_filename: the name of the image file to be created, where the
          faces have polygons drawn around them.
    """
    im = Image.open(image)
    draw = ImageDraw.Draw(im)
    # Sepecify the font-family and the font-size
    for face in faces:
        box = [(vertex.x, vertex.y)
               for vertex in face.bounding_poly.vertices]
        draw.line(box + [box[0]], width=5, fill='#00ff00')
        # Place the confidence value/score of the detected faces above the
        # detection box in the output image
        draw.text(((face.bounding_poly.vertices)[0].x,
                   (face.bounding_poly.vertices)[0].y - 30),
                  str(format(face.detection_confidence, '.3f')) + '%',
                  fill='#FF0000')
    im.save(output_filename)

In [15]:
def main(input_filename, output_filename, max_results):
    with open(input_filename, 'rb') as image:
        faces = detect_face(image, max_results)
        print('Found {} face{}'.format(
            len(faces), '' if len(faces) == 1 else 's'))

        print('Writing to file {}'.format(output_filename))
        # Reset the file pointer, so we can read the file again
        image.seek(0)
        highlight_faces(image, faces, output_filename)

In [18]:
from google.cloud import vision
from PIL import Image, ImageDraw
main("C:/Users/qud97/Desktop/crawl/AutoCrawler/download/ITRI/google_0012.jpg", 
    "C:/Users/qud97/Desktop/crawl/AutoCrawler/google_0012.jpg", 9)

Found 9 faces
Writing to file C:/Users/qud97/Desktop/crawl/AutoCrawler/google_0012.jpg
